In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()

# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns =['EIN', 'NAME'])
application_df.head()

2022-04-10 19:37:42.201596: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-10 19:37:42.201702: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Determine which values to replace if counts are less than ...?
app_counts = application_df.APPLICATION_TYPE.value_counts()
replace_application = list(app_counts[app_counts < 200].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")

# Determine which values to replace if counts are less than ..?
class_counts = application_df.CLASSIFICATION.value_counts()
replace_class = list(class_counts[class_counts < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")

# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)

# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"]
X = application_df.drop(columns = ["IS_SUCCESSFUL"])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

/opt/conda/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


### Deliverable 3: Compile, Train and Evaluate the Model

#### Control Model

In [3]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.iloc[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

import os
from tensorflow.keras.callbacks import ModelCheckpoint

os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4020)

fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])


model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2022-04-10 19:37:45.113118: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-10 19:37:45.113245: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-10 19:37:45.113293: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (8fd7d725f6cb): /proc/driver/nvidia/version does not exist
2022-04-10 19:37:45.114343: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5699 - accuracy: 0.7255
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5556 - accuracy: 0.7316
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5522 - accuracy: 0.7317
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5515 - accuracy: 0.7325
Epoch 5/100
771/804 [===========================>..] - ETA: 0s - loss: 0.5512 - accuracy: 0.7311
Epoch 5: saving model to checkpoints/weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5500 - accuracy: 0.7324
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5496 - accuracy: 0.7328
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5481 - accuracy: 0.7332
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5470 - accuracy: 0.7347
Epoch 9/100
804/804 [==============================] -

766/804 [===========================>..] - ETA: 0s - loss: 0.5378 - accuracy: 0.7386
Epoch 65: saving model to checkpoints/weights.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5378 - accuracy: 0.7394
Epoch 66/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5374 - accuracy: 0.7398
Epoch 67/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5373 - accuracy: 0.7402
Epoch 68/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5376 - accuracy: 0.7407
Epoch 69/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5375 - accuracy: 0.7405
Epoch 70/100
799/804 [============================>.] - ETA: 0s - loss: 0.5368 - accuracy: 0.7405
Epoch 70: saving model to checkpoints/weights.70.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5367 - accuracy: 0.7403
Epoch 71/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5377 - accuracy: 0.7405
Epoch 72/100
804/804 

#### Optimization Attempt 1 - More Neurons

In [4]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.iloc[0])
hidden_nodes_layer1 = 160
hidden_nodes_layer2 = 60

nn = tf.keras.models.Sequential()

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

import os
from tensorflow.keras.callbacks import ModelCheckpoint

os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4020)

fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])


model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5704 - accuracy: 0.7215
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5579 - accuracy: 0.7294
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5530 - accuracy: 0.7305
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5504 - accuracy: 0.7322
Epoch 5/100
783/804 [============================>.] - ETA: 0s - loss: 0.5489 - accuracy: 0.7329
Epoch 5: saving model to checkpoints/weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5494 - accuracy: 0.7324
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5496 - accuracy: 0.7339
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5481 - accuracy: 0.7331
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5473 - accuracy: 0.7331
Epoch 9/100
804/804 [==============================] -

790/804 [============================>.] - ETA: 0s - loss: 0.5367 - accuracy: 0.7413
Epoch 65: saving model to checkpoints/weights.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5370 - accuracy: 0.7409
Epoch 66/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5363 - accuracy: 0.7408
Epoch 67/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5366 - accuracy: 0.7402
Epoch 68/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5361 - accuracy: 0.7412
Epoch 69/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5364 - accuracy: 0.7405
Epoch 70/100
775/804 [===========================>..] - ETA: 0s - loss: 0.5366 - accuracy: 0.7416
Epoch 70: saving model to checkpoints/weights.70.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5359 - accuracy: 0.7416
Epoch 71/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5358 - accuracy: 0.7403
Epoch 72/100
804/804 

#### Optimization Attempt 2 - More Neurons and Additional Hidden Layer

In [5]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.iloc[0])
hidden_nodes_layer1 = 160
hidden_nodes_layer2 = 60
hidden_nodes_layer3 = 30

nn = tf.keras.models.Sequential()

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

import os
from tensorflow.keras.callbacks import ModelCheckpoint

os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4020)

fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])


model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5684 - accuracy: 0.7227
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5562 - accuracy: 0.7309
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5519 - accuracy: 0.7320
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5506 - accuracy: 0.7334
Epoch 5/100
803/804 [============================>.] - ETA: 0s - loss: 0.5494 - accuracy: 0.7332
Epoch 5: saving model to checkpoints/weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5497 - accuracy: 0.7329
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5482 - accuracy: 0.7348
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5481 - accuracy: 0.7332
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5474 - accuracy: 0.7358
Epoch 9/100
804/804 [==============================] -

803/804 [============================>.] - ETA: 0s - loss: 0.5362 - accuracy: 0.7411
Epoch 65: saving model to checkpoints/weights.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5362 - accuracy: 0.7412
Epoch 66/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5356 - accuracy: 0.7406
Epoch 67/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5354 - accuracy: 0.7414
Epoch 68/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5354 - accuracy: 0.7409
Epoch 69/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5363 - accuracy: 0.7409
Epoch 70/100
777/804 [===========================>..] - ETA: 0s - loss: 0.5348 - accuracy: 0.7416
Epoch 70: saving model to checkpoints/weights.70.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5351 - accuracy: 0.7413
Epoch 71/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5353 - accuracy: 0.7418
Epoch 72/100
804/804 

#### Attempt 3 -More Neurons, 3 Added Hidden Layers, and low correlating features removed

In [6]:
asdf = pd.DataFrame(application_df.corr()["IS_SUCCESSFUL"])
asdf["Bob"] = asdf.IS_SUCCESSFUL **2
asdf.sort_values(by="Bob")


,IS_SUCCESSFUL,Bob
CLASSIFICATION_C2000,-0.000434,1.883437e-07
APPLICATION_TYPE_T3,-0.000953,9.091466e-07
STATUS,-0.001636,2.675801e-06
AFFILIATION_Family/Parent,0.002608,6.800676e-06
AFFILIATION_Regional,0.003238,1.048685e-05
AFFILIATION_National,0.004581,2.098632e-05
SPECIAL_CONSIDERATIONS_Y,0.005469,2.991262e-05
SPECIAL_CONSIDERATIONS_N,-0.005469,2.991262e-05
USE_CASE_Other,0.008765,7.682515e-05
INCOME_AMT_5M-10M,-0.009171,8.410324e-05


In [7]:
# Split our preprocessed data into our features and target arrays
#Drop Low Correlating features
y = application_df["IS_SUCCESSFUL"]
X = application_df.drop(columns = ["IS_SUCCESSFUL", "STATUS", "SPECIAL_CONSIDERATIONS_Y", "SPECIAL_CONSIDERATIONS_N", "ASK_AMT"])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.iloc[0])
hidden_nodes_layer1 = 200
hidden_nodes_layer2 = 100
hidden_nodes_layer3 = 50
hidden_nodes_layer4 = 25
hidden_nodes_layer5 = 10

nn = tf.keras.models.Sequential()

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="relu"))

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

import os
from tensorflow.keras.callbacks import ModelCheckpoint

os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4020)

fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])


model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5696 - accuracy: 0.7248
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5546 - accuracy: 0.7303
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5525 - accuracy: 0.7325
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5504 - accuracy: 0.7342
Epoch 5/100
772/804 [===========================>..] - ETA: 0s - loss: 0.5509 - accuracy: 0.7330
Epoch 5: saving model to checkpoints/weights.05.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5493 - accuracy: 0.7343
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5488 - accuracy: 0.7345
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5475 - accuracy: 0.7349
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5466 - accuracy: 0.7371
Epoch 9/100
804/804 [==============================] -

799/804 [============================>.] - ETA: 0s - loss: 0.5356 - accuracy: 0.7401
Epoch 65: saving model to checkpoints/weights.65.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5354 - accuracy: 0.7402
Epoch 66/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5354 - accuracy: 0.7402
Epoch 67/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5349 - accuracy: 0.7406
Epoch 68/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5357 - accuracy: 0.7407
Epoch 69/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5351 - accuracy: 0.7401
Epoch 70/100
786/804 [============================>.] - ETA: 0s - loss: 0.5350 - accuracy: 0.7415
Epoch 70: saving model to checkpoints/weights.70.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5353 - accuracy: 0.7408
Epoch 71/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5349 - accuracy: 0.7411
Epoch 72/100
804/804 

In [9]:
nn.save("AlphabetSoupCharity_Optimization.h5")